## Imports

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

## reading from train_test csv file

In [0]:
data = pd.read_csv("train_test.csv")

با استفاده از دستور
read_csv
فایل ترین را می‌خوانیم.

## split data into train and test dataframes

In [0]:
dataTrain, dataTest = train_test_split(data, test_size=0.2)

## split poems by poets and count number of words for each poet

In [0]:
hafezPoems, saadiPoems, uniqueWords = {}, {}, {}
hafezPoetNum, saadiPoetNum = 0, 0

for i in dataTrain.index:
  if dataTrain["label"][i] == "hafez":
    hafez = dataTrain["text"][i].split()
    hafezPoetNum += 1
    for word in hafez:
      if word in uniqueWords:
        uniqueWords[word] = uniqueWords[word] + 1
      else:
        uniqueWords[word] = 1
      if word in hafezPoems:
        hafezPoems[word] = hafezPoems[word] + 1
      else:
        hafezPoems[word] = 1
  else:
    saadi = dataTrain["text"][i].split()
    saadiPoetNum += 1
    for word in saadi:
      if word in uniqueWords:
        uniqueWords[word] = uniqueWords[word] + 1
      else:
        uniqueWords[word] = 1
      if word in saadiPoems:
        saadiPoems[word] = saadiPoems[word] + 1
      else:
        saadiPoems[word] = 1

با ایتریت کردن روی دیتاستی که خواندیم (دو ستون تکست و لیبل دارد)، تعداد شعرهای حافظ و تعداد شعرهای سعدی و همچنین تعداد تکرار هرکلمه در شعرهای حافظ و تعداد تکرار هرکلمه در شعرهای سعدی، و تعداد کل کلمه‌‌هایی که در کل دیتاست ترین ما قرار دارد را به دست می‌آوریم و در دیکشنری‌های مختلف نگه می‌داریم.

## hafezLikeliHood and saadiLikeliHood

In [0]:
hafezLikeliHood, saadiLikeliHood = {}, {}
hafezWordNum = sum(hafezPoems.values())
saadiWordNum = sum(saadiPoems.values())
for word in hafezPoems:
  hafezLikeliHood[word] = hafezPoems[word] / hafezWordNum
for word in saadiPoems:
  saadiLikeliHood[word] = saadiPoems[word] / saadiWordNum

با جمع کردن تعداد تکرار تمام کلمات به کار رفته در شعرهای حافظ و سعدی به طور جداگانه، تعداد کل کلمات سعدی و حافظ را به دست می‌آوریم. سپس با ایتریت کردن روی تعداد کلمات یونیک حافظ و سعدی و تقسیم تعداد تکرار آن‌ها بر تعداد کل کلمه‌های آن شاعر، احتمال شرطی کلمه به شرط شاعر را به دست می‌آوریم.

## Prior Probability

In [0]:
poetNum = len(dataTrain)
hafezPriorProb = hafezPoetNum / poetNum
saadiPriorProb = saadiPoetNum / poetNum

احتمال پرایور حافظ و سعدی از تقسیم تعداد شعرهای حافظ بر کل تعداد شعرهای داخل دیتاست و تعداد شعرهای سعدی بر کل تعداد شعرهای داخل دیتاست به دست می‌آید.

## Laplace

In [0]:
def laplace(testPoem, alpha):
  hafezProb, saadiProb = 1, 1
  hafezLaplaceProb, saadiLaplaceProb = [], []

  for word in testPoem:
    if word in hafezPoems:
      hafezLaplaceProb.append((hafezPoems[word] + alpha) / ((len(uniqueWords)*alpha) + hafezWordNum))
    else:
      hafezLaplaceProb.append(alpha / ((len(uniqueWords)*alpha) + hafezWordNum))
  for prob in hafezLaplaceProb:
    hafezProb *= prob

  for word in testPoem:
    if word in saadiPoems:
      saadiLaplaceProb.append((saadiPoems[word] + alpha) / ((len(uniqueWords)*alpha) + saadiWordNum))
    else:
      saadiLaplaceProb.append(alpha / ((len(uniqueWords)*alpha) + saadiWordNum))
  for prob in saadiLaplaceProb:
    saadiProb *= prob
  
  return hafezProb, saadiProb


در حالتی که یکی از شاعران کلمه‌ای که در بیت تست می‌آید را کلا ندیده باشد و در دیتاستش قرار نداشته باشد از بهبود‌دهنده‌ی لاپلاس استفاده می‌کنیم. در حالت قبل در این حالت‌ها احتمال شرطی تکرار کلمه به شرط شاعر را صفر در نظر می‌گرفتیم. اما حالا برای اینکه آن را بهتر کنیم تعداد تکرار این کلمات در دیتاست آن شاعر را آلفا درنظر می‌گیریم. آلفا را عدد کوچکی می‌دهیم که عملا در دیتاست و احتمال بقیه‌ی کلمات تاثیری نگذارد.
حال برای آنکه بتوانیم احتمال پوستریر هر شاعر را حساب کنیم نیاز است که احتمال جمع تمام این احتمال‌های شرطی یک شوند. بنابراین باید احتمال بقیه‌ی کلمات داخل دیتاست را هم تغییر دهیم. درواقع به تمام کلمات داخل دیتاست هم مقدار آلفا اضافه می‌کنیم و دوباره احتمال پوستریر شعرا را حساب می‌کنیم و با یکدیگر مقایسه می‌کنیم.

##Without Laplace

In [0]:
def withoutLaplace(testPoem):
  hafezProb, saadiProb = 1, 1
  for word in testPoem:
    if word not in hafezPoems and word not in saadiPoems:
      continue
    if word in hafezPoems:
      hafezProb *= hafezLikeliHood[word]
    else:
      hafezProb = 0
    if word in saadiPoems:
      saadiProb *= saadiLikeliHood[word]
    else:
      saadiProb = 0

زمانی که بهبود‌دهنده‌ی لاپلاس را نداریم، به ازای هر بیتی که در دیتاست تست قرار دارد، کلمه‌هایش را درمیاوریم. اگر کلمه نه در کلمه‌های دیده شده‌ی حافظ قرار داشته باشد و نه در کلمه‌های دیده شده‌ی سعدی، آن کلمه را ایگنور می‌کنیم و از آن می‌گذریم.
اما اگر کلمه در بیت تست در هرکدام از کلمه‌های دیده شده‌ی دو شاعر قرار داشته باشد، احتمال شرطی آن کلمه به شرط شاعرش را از روی دیکشنری‌ای که قبلا خوانده بودیم به دست می‌آوریم.
به ازای تمام کلمات توی بیت تست این کار را هم برای سعدی و هم برای حافظ انجام می‌دهیم. سپس این احتمال‌ها را در هم ضرب می‌کنیم.
همچنین اگر کلمه‌ای در دیتاست یکی از شاعران نباشد، احتمال آن را صفر در نظر می‌گیریم.

## Probability

In [0]:
def prediction(dfTest, isEval, alpha):
  if isEval:
    with open('output.csv', mode='w') as out:
      out = csv.writer(out, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
      out.writerow(["id", "ppoet"])
  else:
    correctHafez, allHafez, detectedHafez, correctDetected, total = 0, 0, 0, 0, 0
    total = len(dfTest)
    for i in dfTest.index:
      if dfTest["label"][i] == "hafez":
        allHafez += 1
      
  for i in dfTest.index:
    testPoem = dfTest["text"][i].split()
    
    if alpha > 0:
      hafezProb, saadiProb = laplace(testPoem, alpha)
    else:
      hafezProb, saadiProb = withoutLaplace(testPoem)
    
    hafezProb *= hafezPriorProb
    saadiProb *= saadiPriorProb

    if isEval:
      pred = ""
      if hafezProb >= saadiProb:
        pred = "hafez"
      else:
        pred = "saadi"
      out.writerow([i, pred])
    else:
      if hafezProb >= saadiProb:
        detectedHafez += 1
        if "hafez" == dfTest["label"][i]:
          correctHafez += 1
          correctDetected += 1
      elif saadiProb > hafezProb:
        if "saadi" == dfTest["label"][i]:
          correctDetected += 1

  if not isEval:
    recall = correctHafez / allHafez
    precision = correctHafez / detectedHafez
    accuracy = correctDetected / total

    print("recall: ", recall)
    print("precision: ", precision)
    print("accuracy: ", accuracy)

## Test

In [34]:
isEval = 0
alpha = 0.3
prediction(dataTest, isEval, alpha)

recall:  0.8265486725663717
precision:  0.8839116719242902
accuracy:  0.8855911919578746


## Evaluate

In [0]:
evalTest = pd.read_csv("evaluate.csv")
isEval = 1
alpha = 0.3
prediction(evalTest, isEval, alpha)

در این پروژه تعداد تکرار هر کلمه در شعرهای شعرا را به عنوان فیچر در نظر گرفتیم. سوالی که در پروژه مطرح است این است که احتمال اینکه شاعر یک شعر حافظ است یا سعدی به شرط آن که کلمه‌های داخل آن چه باشد. برای اینکه بتوانیم این احتمال را بررسی کنیم از قانون احتمال بیزی استفاده می‌کنیم. به این صورت که ابتدا احتمال هر فیچر که در اینجا تعداد تکرار کلمه‌ها در شعر شعرا است را به شرط هر شاعر حساب می‌کنیم. چون از ناییو بیز استفاده می‌کنیم شرایط را ساده تر فرض می‌کنیم. درواقع کلمات را مستقل از یکدیگر درنظر می‌گیریم. بنابراین این احتمال‌ها ضرب می‌شوند. سپس احتمال پرایور را حساب می‌کنیم که این همان تعداد شعرهای حافظ به کل شعرهای آمده شده در دیتاست ترین است. همچنین در مخرج هم احتمال آمدن یک کلمه را قرار می‌دهیم. یعنی تعداد تکرار یک کلمه در کل دیتاست ترین. 
همانطور که در راهنمایی هم آمده است برای مقایسه‌ی بین اینکه احتمال کدام شاعر بیشتر است از مخرج صرفه نظر می‌کنیم.

سوال اضافه اول:

precision = true positive / (true positive + false positive)

با توجه به فرمول بالا، پرسیژن اگر بالا باشد به این معنی است که فالس پازتیو عدد کمی است. به این معنی که تعداد بیت‌هایی که ما حافظ تشخیص دادیم اما واقعا حافظ نبوده‌اند کم بوده است.
فرض کنید در یک دیتاستی ما یک بیت شعر را به درستی حافظ تشخیص داده‌ایم و بقیه را سعدی پیشگویی کرده‌ایم. در این حالت پرسیژن یک خواهد بود اما مدلی که ایجنت ما پیشبینی می‌کند، مدل خوبی نیست.*italicized text*


سوال اضافه دوم:

در مسائل 

unbalanced classification

داده‌های یک کلاس خیلی بیشتر از یک کلاس دیگر هستند.

accuracy = (true positive + true negative) / (true positive + false positive + true negative + false negative)

با توجه به نعریف بالا زمانی اکیورسی بالایی می‌گیریم که تمام نمونه‌ها را در آن کلاسی که تعداد داده‌هایش بیشتر است قرار دهیم. در اینجا مثال سرطان داشتن بیان می‌شود. به عنوان مثال ۹۰٪ سرطان دارند و ۱۰٪ سرطان ندارند. در این حالت اگر ایجنت ما احتمال دهد که یک فرد جدید سرطان دارد، اکیورسی بالایی می‌گیرد.با اینکه واضح است که این مدل، مدل خوبی نیست  و عملا تشخیصی انجام نمی‌دهد. در این حالت بهتر است که اکیورسی پایین‌تری داشته باشیم.